In [82]:
# inicia bibliotecas e local do arquivo
import pandas as pd
import numpy as np
df = pd.read_csv('./vendas_modificado.csv')
df.head(20)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,"R$ 10,16",1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,Feijão,"R$ 7,77",3,43.31,Em Separação,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,Queijo Mussarela,"R$ 14,93",14,218.02,Em Separação,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,Condicionador,"R$ 9,51",5,69.55,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,Sabonete,"R$ 1,25",2,13.00,Em Separação,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove


In [83]:
#verifica os tipos
df.dtypes


id_da_compra      int64
data             object
hora             object
cliente          object
produto          object
valor            object
quantidade        int64
total           float64
status           object
cidade           object
estado           object
pais             object
cep              object
frete           float64
pagamento        object
vendedor         object
marca            object
dtype: object

In [84]:
# verifica nulos
df.isnull().sum()

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3685
status             0
cidade             0
estado             0
pais               0
cep                0
frete           7371
pagamento          0
vendedor        3680
marca              0
dtype: int64

In [85]:
# verifica os erros de digitação na coluna valor
erros_valor = df[~df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').str.strip().str.match(r'^\d+(\.\d+)?$')]


print(erros_valor[['valor']].head(10))


                                                   valor
394    R$ 11,0111,0111,0111,0111,0111,0111,0111,0111,...
7718   R$ 3,133,133,133,133,133,133,133,133,133,133,1...
12960  R$ 3,543,543,543,543,543,543,543,543,543,543,5...
15553  R$ 2,472,472,472,472,472,472,472,472,472,472,4...
15575  R$ 2,162,162,162,162,162,162,162,162,162,162,1...
18363  R$ 9,889,889,889,889,889,889,889,889,889,889,8...
21016  R$ 4,474,474,474,474,474,474,474,474,474,474,4...
31026  R$ 7,617,617,617,617,617,617,617,617,617,617,6...
31176  R$ 8,948,948,948,948,948,948,948,948,948,948,9...
33643  R$ 2,212,212,212,212,212,212,212,212,212,212,2...


In [86]:
# corige os erros dos valores
df = df[df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').str.strip().str.match(r'^\d+(\.\d+)?$')]

df['valor'] = df['valor'].str.replace('R$', '', regex=False).str.replace(',', '.').astype(float)



In [87]:
#padroniza data hora e frete
df['data'] = pd.to_datetime(df['data'], errors='coerce')


df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.strftime('%H:%M:%S')

df['frete'] = df['frete'].fillna(0.0)

In [88]:
# verifica nulos
df.isnull().sum()

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3684
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3680
marca              0
dtype: int64

In [89]:
# corrige os totais que estão com o valor de Nan com base na formula e verifica se ainda existem nulos 
# como não existem valores nem quantidades nulas não é feita nenhuma verificação
# Fórmula: total = valor * quantidade + frete
#faltando_total = df['total'].isna()
#df.loc[faltando_total, 'total'] = df.loc[faltando_total, 'valor'] * df.loc[faltando_total, 'quantidade'] + df.loc[faltando_total, 'frete']

df.isnull().sum()

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total           3684
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor        3680
marca              0
dtype: int64

In [90]:
# dropa os nulos restantes 
df = df.dropna()


In [91]:
# Corrige os valores nulos na coluna 'total' usando a fórmula correta
faltando_total = df['total'].isna()
df.loc[faltando_total, 'total'] = (df.loc[faltando_total, 'valor'] * df.loc[faltando_total, 'quantidade'] + df.loc[faltando_total, 'frete']).round(2)

# Recalcula todos os valores de 'total' e padroniza com duas casas decimais
df['total'] = (df['valor'] * df['quantidade'] + df['frete']).round(2)

# Calcula 'total_calculado' como referência de verificação
df['total_calculado'] = (df['valor'] * df['quantidade'] + df['frete']).round(2)

# Verifica inconsistências entre 'total' e 'total_calculado'
df_totais_errados = df[~np.isclose(df['total'], df['total_calculado'])]

# Exibe registros com discrepâncias
print(df_totais_errados[['valor', 'quantidade', 'frete', 'total', 'total_calculado']])


Empty DataFrame
Columns: [valor, quantidade, frete, total, total_calculado]
Index: []


In [ ]:
df = df[df['total'] <= 999]

In [92]:
# exibe o data frame
df.head(30)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,239.31
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,9.75
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,36.67
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás,54.96
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações,18.96
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue com Sucesso,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações,20.16
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,Feijão,7.77,3,43.31,Em Separação,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom,43.31
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,Queijo Mussarela,14.93,14,218.02,Em Separação,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre,218.02
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,Condicionador,9.51,5,69.55,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene,69.55
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,Sabonete,1.25,2,13.00,Em Separação,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove,13.00


In [93]:
# Verificando os status 
df['status'].value_counts()

status
Em Separação            105154
Pagamento Confirmado    104333
Aguardando Pagamento     71265
Em Transporte            35048
Entregue                 34675
PC                        1687
Sep                       1633
Pgto Confirmado           1597
Separando                 1589
Aguardando Pgto            803
aguardando pagamento       744
AP                         682
Entg                       559
Transp                     552
Entregue com Sucesso       525
Transportando              494
Name: count, dtype: int64

In [94]:
df.isnull().sum()

id_da_compra       0
data               0
hora               0
cliente            0
produto            0
valor              0
quantidade         0
total              0
status             0
cidade             0
estado             0
pais               0
cep                0
frete              0
pagamento          0
vendedor           0
marca              0
total_calculado    0
dtype: int64

In [95]:
# Corrigindo e padronizando os nomes dos status
df['status'] = df['status'].replace({
    'Pgto Confirmado': 'Pagamento Confirmado',
    'aguardando pagamento': 'Aguardando Pagamento',
    'Aguardando Pgto': 'Aguardando Pagamento',
    'AP': 'Aguardando Pagamento',
    'Sep': 'Em Separação',
    'Separando': 'Em Separação',
    'PC': 'Pagamento Confirmado',
    'Entg': 'Entregue',
    'Entregue com Sucesso': 'Entregue',
    'Transp': 'Em Transporte',
    'Transportando': 'Em Transporte'
})


In [96]:
# Verificando como ficou a distribuição dos status após a padronização
df['status'].value_counts()


status
Em Separação            108376
Pagamento Confirmado    107617
Aguardando Pagamento     73494
Em Transporte            36094
Entregue                 35759
Name: count, dtype: int64

In [97]:
df['pagamento'].value_counts()

pagamento
Cartão de Crédito         106293
Transferência Bancária    104548
Pix                       104363
Boleto                     35300
Pix à Vista                 1682
Pagamento Instantâneo       1653
Cartão Crédito à Vista      1642
Cartão Crédito              1617
TED                         1592
DOC                         1560
Boleto à Vista               580
Boleto no Dinheiro           510
Name: count, dtype: int64

In [98]:
# verificando produtos
print(df['produto'].value_counts().to_string())


produto
Pasta de Dente           25184
Queijo Mussarela         25120
Sabonete                 23776
Manteiga                 23311
Café                     21466
Açúcar                   21233
Papel Toalha             18075
Desinfetante             17806
Molho de Tomate          12026
Condicionador            10908
Refrigerante              9258
Água Mineral              9202
Farinha de Trigo          9184
Macarrão                  9137
Sal                       9125
Presunto                  9116
Vinho                     9105
Shampoo                   9099
Biscoito Recheado         9094
Detergente                9085
Óleo de Soja              8998
Suco de Laranja           8883
Cerveja                   8258
Pão de Forma              6526
Feijão                    6069
Arroz                     6037
Papel Higiênico           5259
Amaciante                 4769
Leite Integral            4555
Sabão em Pó               2880
Carvão                    1528
Pasta de Dente#$@!         530


In [99]:
# Dicionário de correções: chaves = nomes errados / valores = nomes corretos
correcoes_produtos = {
    # Corrigindo com símbolos e variações comuns
    'Pasta de Dente#$@!': 'Pasta de Dente',
    'Queijo Mussarela#$@!': 'Queijo Mussarela',
    'Sabonete#$@!': 'Sabonete',
    'Manteiga#$@!': 'Manteiga',
    'Café#$@!': 'Café',
    'Açúcar#$@!': 'Açúcar',
    'Desinfetante#$@!': 'Desinfetante',
    'Papel Toalha#$@!': 'Papel Toalha',
    'Condicionador#$@!': 'Condicionador',
    'Molho de Tomate#$@!': 'Molho de Tomate',
    'Água Mineral#$@!': 'Água Mineral',
    'Refrigerante#$@!': 'Refrigerante',
    'Presunto#$@!': 'Presunto',
    'Cerveja#$@!': 'Cerveja',
    'Biscoito Recheado#$@!': 'Biscoito Recheado',
    'Macarrão#$@!': 'Macarrão',
    'Óleo de Soja#$@!': 'Óleo de Soja',
    'Vinho#$@!': 'Vinho',
    'Sal#$@!': 'Sal',
    'Suco de Laranja#$@!': 'Suco de Laranja',
    'Shampoo#$@!': 'Shampoo',
    'Farinha de Trigo#$@!': 'Farinha de Trigo',
    'Detergente#$@!': 'Detergente',
    'Pão de Forma#$@!': 'Pão de Forma',
    'Arroz#$@!': 'Arroz',
    'Feijão#$@!': 'Feijão',
    'Amaciante#$@!': 'Amaciante',
    'Leite Integral#$@!': 'Leite Integral',
    'Papel Higiênico#$@!': 'Papel Higiênico',
    'Sabão em Pó#$@!': 'Sabão em Pó',
    'Carvão#$@!': 'Carvão',
    'Condicioiador#$@!': 'Condicionador',

    # Corrigindo erros de digitação
    'Refrigkrante': 'Refrigerante',
    'Scl': 'Sal',
    'Papel qoalha': 'Papel Toalha',
    'Desinfetanue': 'Desinfetante',
    'Qbeijo Mussarela': 'Queijo Mussarela',
    'Água Mineras': 'Água Mineral',
    'Presuntd': 'Presunto',
    'Macawrão': 'Macarrão',
    'Sucoyde Laranja': 'Suco de Laranja',
    'Clfé': 'Café',
    'Desinfekante': 'Desinfetante',
    'Majarrão': 'Macarrão',
    'Cnfé': 'Café',
    'Farinha de Tripo': 'Farinha de Trigo',
    'Manteigt': 'Manteiga',
    'zabonete': 'Sabonete',
    'Mqcarrão': 'Macarrão',
    'Arroc': 'Arroz',
    'tal': 'Sal',
    'Sabonepe': 'Sabonete',
    'Papel Twalha': 'Papel Toalha',
    'Água Mineual': 'Água Mineral',
    'Condibionador': 'Condicionador',
    'Caft': 'Café',
    'ieijão': 'Feijão',
    'Açúcaz': 'Açúcar',
    'Amaciayte': 'Amaciante',
    'Caff': 'Café',
    'Mopho de Tomate': 'Molho de Tomate',
    'Molmo de Tomate': 'Molho de Tomate',
    'Biscoitq Recheado': 'Biscoito Recheado',
    'Macirrão': 'Macarrão',
    'Deqergente': 'Detergente',
    'Cafc': 'Café',
    'Queijo Mussarelz': 'Queijo Mussarela',
    'Deterwente': 'Detergente',
}

# Aplicando as correções no DataFrame
df['produto'] = df['produto'].replace(correcoes_produtos)


In [100]:
# dropa duplicatas
df = df.drop_duplicates()


In [101]:
# verificando cidades
df['cidade'].value_counts()

cidade
Cabo Frio                6985
Astolfo Dutra            6686
Palma                    6648
Volta Redonda            6645
Mariana                  6637
Macaé                    6556
Guarulhos                6539
São João de Meriti       6521
Viçosa                   6519
Rio de Janeiro           6516
Petrópolis               6514
Lima Duarte              6501
Carandaí                 6479
Lagoa Dourada            6459
Barbacena                6437
Juiz de Fora             6426
Cataguases               6419
Ouro Preto               6415
Campinas                 6407
Congonhas                6392
São Vicente de Minas     6386
Coronel Xavier Chaves    6373
Itaboraí                 6371
Duque de Caxias          6365
Ritápolis                6365
Angra dos Reis           6357
Campos dos Goytacazes    6351
Tiradentes               6346
Prados                   6343
Conselheiro Lafaiete     6341
Tocantins                6338
Matias Barbosa           6334
Santos Dumont            6333
Nit

In [102]:
# verificando pais
df['pais'].value_counts()

pais
Brasil    355757
Name: count, dtype: int64

In [103]:
# verificando marcas
print(df['marca'].value_counts().to_string())


marca
Dove                 14707
Itambé                9208
Quatá                 8512
Colgate               8491
Porto Alegre          8489
Oral-B                8433
Sorriso               8368
Tirolez               8246
Lux                   8086
Aviação               7892
Nivea                 7890
Qualy                 7832
Itamarati             7263
3 Corações            7254
Melitta               7196
Pilão                 7113
União                 7056
Caravelas             7014
Seda                  6712
Pantene               6615
Elite                 6084
Snob                  6035
Scott                 5977
Veja                  5977
Pinho Sol             5957
Lysoform              5949
Predilecta            4111
Fugini                4024
Elefante              3944
Fanta                 3146
Bauducco              3145
São Lourenço          3127
Sol                   3125
Minalba               3121
Perdigão              3120
Guaraná Antártica     3116
Miolo                 

In [104]:

# verificando clientes
print(df['cliente'].value_counts().to_string())

cliente
VICTOR HENRIQUE CAÇADOR                       2472
JOÃO QUEIROZ DE AGUIAR                        1691
SAMUEL VITOR FAÇANHA NEVES                    1667
VICTOR SILVINO DA SILVA                       1642
MATEUS ANTÔNIO DE DECCO OLIVEIRA              1639
MATHEUS AUGUSTO NERY SILVA                    1622
GABRIEL SILVA RAMOS                           1606
MATHEUS GONÇALVES DOS SANTOS                  1567
FELIPE DA SILVA BIANQUINI                     1536
GABRIEL MATOS LIMA DA CUNHA                   1531
JOÃO DA CUNHA TEIXEIRA                        1467
GABRIEL MATHEUS DA SILVA                      1459
GABRIEL VICTOR ALVES                          1411
SAMUEL RIBEIRO AMORIM                         1402
GABRIEL GONÇALVES DOS SANTOS                  1401
GABRIEL SOUZA RONCETE                         1376
GABRIEL SEGURASSE GONÇALVES                   1361
GABRIEL CARLOS DE SOUZA                       1259
JOÃO MATHEUS DA SILVA                         1258
GABRIEL CASTILHO CARDOS

In [105]:
# exibe o data frame
df.head(50)

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca,total_calculado
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.0,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre,239.31
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.0,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini,9.75
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba,36.67
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.0,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás,54.96
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.0,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações,18.96
5,38623,2018-02-19,17:32:01,LUCAS ANTÔNIO DE SOUZA NETO,Café,10.16,1,20.16,Entregue,Resende Costa,MG,Brasil,36340-000,10.0,Transferência Bancária,VICTOR GONÇALVES DONADONI,3 Corações,20.16
6,46627,2018-04-21,22:08:30,GUILHERME IVAIR DE ASSIS SETTA,Feijão,7.77,3,43.31,Em Separação,São Paulo,SP,Brasil,01000-000,20.0,Pix,SAMUEL HENRIQUE CAÇADOR,Caldo Bom,43.31
7,53579,2020-09-25,12:04:15,GABRIEL MUZY FERNANDES,Queijo Mussarela,14.93,14,218.02,Em Separação,Bicas,MG,Brasil,36600-000,9.0,Boleto,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Porto Alegre,218.02
8,34924,2018-06-20,17:10:42,LEVI MATOS LIMA DA CUNHA,Condicionador,9.51,5,69.55,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.0,Pix,FELIPE HENRIQUE COSTA BARNABE MARAZO,Pantene,69.55
9,10575,2018-10-01,18:36:45,HENRICO DA CUNHA TEIXEIRA,Sabonete,1.25,2,13.00,Em Separação,São João del-Rei,MG,Brasil,36300-000,10.5,Boleto,GABRIEL QUEIROZ DE AGUIAR,Dove,13.00


In [106]:
df.isnull().sum().loc[lambda x: x > 0]

Series([], dtype: int64)

In [107]:
df['valor'] = df['valor'].round(2)
df['frete'] = df['frete'].round(2)
df['total'] = df['total'].round(2)

print(df['valor'])

0          16.87
1           3.25
2           1.63
3           8.74
4           9.48
           ...  
368747      1.43
368748    682.51
368749     11.01
368750      4.56
368751      3.17
Name: valor, Length: 355757, dtype: float64


In [108]:
df.to_csv('vendas_tratado.csv', index=False)


Relatório de Limpeza e Padronização de Dados 
Durante a análise e tratamento do conjunto de dados, foram identificados e corrigidos diversos problemas que comprometiam a integridade e a qualidade das informações. Abaixo estão as principais etapas realizadas:

1. Leitura e Inspeção Inicial
Foi realizada a importação do arquivo vendas_modificado.csv.

Analisaram-se os tipos de dados e valores nulos.

2. Tratamento da Coluna valor
Identificados erros de digitação e formatação na coluna valor, como presença de símbolos ("R$") e vírgulas como separador decimal.

As entradas inválidas foram removidas.

Os valores foram convertidos corretamente para o tipo float.

3. Padronização de Datas, Horários e Frete
A coluna data foi convertida para o formato datetime.

A coluna hora foi formatada para o padrão HH:MM:SS.

A coluna frete teve valores nulos preenchidos com 0.0.

4. Correção da Coluna total
Valores ausentes em total foram recalculados usando a fórmula:
total = valor * quantidade + frete.

Valores incorretos foram ajustados com base na mesma fórmula.

5. Eliminação de Duplicatas e Valores Faltantes
Registros com valores ainda nulos foram removidos.

Também foram eliminadas duplicatas do DataFrame.

6. Padronização dos Status de Pagamento
A coluna status apresentava inconsistência nos nomes.

Os valores foram padronizados para categorias claras como:
Pagamento Confirmado, Aguardando Pagamento, Em Separação, Em Transporte, Entregue, entre outros.

7. Correção de Nomes de Produtos
Diversos produtos apresentavam caracteres especiais, erros de digitação ou variações no nome.

Foi criado um dicionário de mapeamento para corrigir e padronizar todos esses valores.

8. Análises Finais
Foi realizada uma checagem nas colunas de cidade, país, marca e cliente para garantir consistência nos dados.

Resumo
O processo de limpeza resultou em um DataFrame com:

Tipos de dados consistentes

Nomes padronizados

Cálculos corretos

Ausência de valores nulos e duplicados

Essas correções garantem maior confiabilidade para análises futuras com base nesse conjunto de dados.